In [0]:
from pyspark.sql.functions import count_distinct, approx_count_distinct, skewness, avg, stddev_samp, sum

# Lab #6 spark dataframes and spark sql
> Due Monday Feb 23rd
(We will be using Databricks community edition for this lab)

Export as an IPython notebook and submit to the dropbox

1. Load the flight-summary.csv into a dataframe: ( be sure to use the correct path )


In [0]:
flight_summary = spark.read \
.format("csv") \
.option("header", "true") \
.option("inferSchema", "true") \
.load("dbfs:/FileStore/tables/flight_summary.csv")

This data represents the number of flights taking place from origin airports to destination airports.

2. Write a query which determines how many unique origin airports are contained in the
data. ( 3 marks)

In [0]:
flight_summary.show()

+-----------+--------------------+---------------+------------+---------+--------------------+----------------+----------+-----+
|origin_code|      origin_airport|    origin_city|origin_state|dest_code|        dest_airport|       dest_city|dest_state|count|
+-----------+--------------------+---------------+------------+---------+--------------------+----------------+----------+-----+
|        BQN|Rafael Hernández ...|      Aguadilla|          PR|      MCO|Orlando Internati...|         Orlando|        FL|  441|
|        PHL|Philadelphia Inte...|   Philadelphia|          PA|      MCO|Orlando Internati...|         Orlando|        FL| 4869|
|        MCI|Kansas City Inter...|    Kansas City|          MO|      IAH|George Bush Inter...|         Houston|        TX| 1698|
|        SPI|Abraham Lincoln C...|    Springfield|          IL|      ORD|Chicago O'Hare In...|         Chicago|        IL|  998|
|        SNA|John Wayne Airpor...|      Santa Ana|          CA|      PHX|Phoenix Sky Harbo...|   

In [0]:
flight_summary.select(count_distinct("origin_airport")).show()

+------------------------------+
|count(DISTINCT origin_airport)|
+------------------------------+
|                           322|
+------------------------------+



3. Modify query #2 to use the approx._count_distinct function with a margin of error of of 10%. (3 marks) Why does this function exists as it is not completely accurate? ( 2 marks)

In [0]:
flight_summary.select(approx_count_distinct("origin_airport", 0.1)).show()

+-------------------------------------+
|approx_count_distinct(origin_airport)|
+-------------------------------------+
|                                  346|
+-------------------------------------+



this function exists to get results with an accuracy based on acceptable deviation to act faster and consume a lot less memory. which is useful for very large datasets and in instances where cost is a factor

4. What does the function skewness determine? (2 marks) Write a query which outputs the skewness of the “count” column. (3 marks) What does the result indicate? ( 2 marks )

The skewness function measures the asymmetry or lack of symmetry in the distribution of a dataset.

A positive skewness indicates that the data's tail is skewed towards the right, meaning the right tail is longer or fatter than the left.
A negative skewness indicates that the tail is skewed towards the left.
A skewness close to 0 indicates that the data distribution is symmetrical.

In [0]:
flight_summary.select(skewness("count")).show()

+-----------------+
|  skewness(count)|
+-----------------+
|2.682183800064101|
+-----------------+



the skewness of the count function indicated that the right tail is longer (or fatter) than the left.
this indicated that there are fewer aiports with a *much* higher count than the larger number of airports with an average to low count

5. Write a query which outputs the top 5 most popular destination cities. You will need to group the data by destination state and destination city. ( 5 marks )

In [0]:
flight_summary.createOrReplaceTempView("flight_summary")
spark.sql("Select sum(Count) as count, dest_state, dest_city from flight_summary group by dest_state, dest_city order by count desc limit 5").show()

+------+----------+-----------------+
| count|dest_state|        dest_city|
+------+----------+-----------------+
|366790|        IL|          Chicago|
|346904|        GA|          Atlanta|
|239582|        TX|Dallas-Fort Worth|
|198724|        TX|          Houston|
|196010|        CO|           Denver|
+------+----------+-----------------+



6. Write a query which groups the data by each origin airport and outputs the sum, average and standard deviation of the count column. Use the “.agg” function ( 5 marks )

In [0]:
flight_summary.groupBy("origin_airport").agg(sum("count"), avg("count"), stddev_samp("count")).show()

+--------------------+----------+------------------+------------------+
|      origin_airport|sum(count)|        avg(count)|stddev_samp(count)|
+--------------------+----------+------------------+------------------+
|Melbourne Interna...|      1332|            1332.0|              null|
|San Diego Interna...|     70207|1526.2391304347825| 1575.804150052814|
|     Eppley Airfield|     16753| 797.7619047619048| 688.7479876385778|
|     Kahului Airport|     20627|1145.9444444444443|1899.6718438664834|
|Austin-Bergstrom ...|     42067|1026.0243902439024|1100.5043954434002|
|Port Columbus Int...|     24187| 863.8214285714286| 823.0782804934813|
|Waco Regional Air...|      1612|            1612.0|              null|
|Sacramento Intern...|     37212|1378.2222222222222|1327.6026204963282|
|Brownsville/South...|      2362|            1181.0| 172.5340546095176|
|       Meadows Field|      2637|            659.25| 160.1527916293271|
|Erie Internationa...|       746|             373.0| 374.7665940